In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from sklearn.feature_selection import mutual_info_classif

mi_scores= mutual_info_classif(X_sm_tl, y_sm_tl)

In [8]:
feature_names = X.columns
mi_df= pd.DataFrame({'Feature': feature_names,
                     'Mutual Information': mi_scores})

mi_df= mi_df.sort_values(by='Mutual Information', ascending=False)
mi_df.head(10)

,Feature,Mutual Information
9,Exercise Hours Per Week,0.117371
14,Sedentary Hours Per Day,0.114291
19,Sleep Hours Per Day,0.111938
3,Heart Rate,0.110179
0,Age,0.108590
18,Physical Activity Days Per Week,0.107702
21,LBP,0.106584
16,BMI,0.099527
13,Stress Level,0.093550
10,Diet,0.090261


In [9]:
k= 10
top_k_features= mi_df.head(k)['Feature'].values
top_k_indices= [feature_names.get_loc(f) for f in top_k_features]

X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.62      0.49      0.55      1125
           1       0.34      0.47      0.39       628

    accuracy                           0.48      1753
   macro avg       0.48      0.48      0.47      1753
weighted avg       0.52      0.48      0.49      1753

Precision: 0.33867276887871856
Accuracy: 0.48088990302338847
AUC: 0.4742491153573956


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.63      0.43      0.51      1125
           1       0.35      0.56      0.43       628

    accuracy                           0.47      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.47      0.48      1753

Precision: 0.352112676056338
Accuracy: 0.4740444951511694
AUC: 0.4924401981599434


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.5991126621804719


In [13]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.63      0.39      0.49      1125
           1       0.35      0.59      0.44       628

    accuracy                           0.46      1753
   macro avg       0.49      0.49      0.46      1753
weighted avg       0.53      0.46      0.47      1753

Precision: 0.35294117647058826
Accuracy: 0.46491728465487736
AUC: 0.49471337579617836


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.66      0.58      0.62      1125
           1       0.38      0.45      0.41       628

    accuracy                           0.54      1753
   macro avg       0.52      0.52      0.51      1753
weighted avg       0.56      0.54      0.54      1753

Precision: 0.376657824933687
Accuracy: 0.5356531660011409
AUC: 0.5162512384996462


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 16, 'weights': 'uniform'}
Best Precision Score: 0.543191741323317


In [16]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.65      0.58      0.61      1125
           1       0.37      0.43      0.40       628

    accuracy                           0.53      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.53      0.54      1753

Precision: 0.3674293405114401
Accuracy: 0.5293782087849401
AUC: 0.5157317763623496


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.63      0.50      0.56      1125
           1       0.34      0.46      0.39       628

    accuracy                           0.49      1753
   macro avg       0.49      0.48      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.34312573443008226
Accuracy: 0.4894466628636623
AUC: 0.4812285916489738


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.65      0.49      0.56      1125
           1       0.37      0.53      0.44       628

    accuracy                           0.50      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.50      0.51      1753

Precision: 0.3673245614035088
Accuracy: 0.5037079292641187
AUC: 0.5098605803255485


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.65      0.20      0.31      1125
           1       0.36      0.80      0.50       628

    accuracy                           0.42      1753
   macro avg       0.50      0.50      0.40      1753
weighted avg       0.55      0.42      0.38      1753

Precision: 0.35994297933000713
Accuracy: 0.41756988020536223
AUC: 0.49518259023354566


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.6304345722333574


In [22]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.67      0.18      0.28      1125
           1       0.36      0.84      0.51       628

    accuracy                           0.42      1753
   macro avg       0.52      0.51      0.39      1753
weighted avg       0.56      0.42      0.36      1753

Precision: 0.3633860977288369
Accuracy: 0.4152880775812892
AUC: 0.5040389242745931


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.63      0.16      0.26      1125
           1       0.36      0.83      0.50       628

    accuracy                           0.40      1753
   macro avg       0.49      0.50      0.38      1753
weighted avg       0.53      0.40      0.34      1753

Precision: 0.3558743169398907
Accuracy: 0.40102681118083283
AUC: 0.5060962491153573


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6239589070988775


In [26]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.54      0.40      0.32      1753

Precision: 0.3587385019710907
Accuracy: 0.3964632059326868
AUC: 0.5083708421797594


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.58      0.08      0.14      1125
           1       0.35      0.90      0.51       628

    accuracy                           0.37      1753
   macro avg       0.47      0.49      0.32      1753
weighted avg       0.50      0.37      0.27      1753

Precision: 0.35286783042394015
Accuracy: 0.3725042783799201
AUC: 0.49994762915782026


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6622589773588186


In [29]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.12      0.21      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.39      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.54      0.39      0.31      1753

Precision: 0.357421875
Accuracy: 0.39189960068454077
AUC: 0.5054210898796886


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.64      0.23      0.34      1125
           1       0.36      0.77      0.49       628

    accuracy                           0.42      1753
   macro avg       0.50      0.50      0.41      1753
weighted avg       0.54      0.42      0.39      1753

Precision: 0.3576208178438662
Accuracy: 0.4232743867655448
AUC: 0.5044515215852795


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.6628530359763335
              precision    recall  f1-score   support

           0       0.66      0.17      0.28      1125
           1       0.36      0.84      0.51       628

    accuracy                           0.41      1753
   macro avg       0.51      0.51      0.39      1753
weighted avg       0.55      0.41      0.36      1753

Precision: 0.36195054945054944
Accuracy: 0.41243582430119796
AUC: 0.4992413305024769


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038061 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1193
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.65      0.18      0.28      1125
           1       0.36      0.82      0.50       628

    accuracy                           0.41      1753
   macro avg       0.51      0.50      0.39      1753
weighted avg       0.55      0.41      0.36      1753

Precision: 0.36022253129346316
Accuracy: 0.41243582430119796
AUC: 0.4988237791932059


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1193
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.020521
0:	learn: 0.6848187	total: 159ms	remaining: 2m 38s
1:	learn: 0.6768108	total: 171ms	remaining: 1m 25s
2:	learn: 0.6731455	total: 185ms	remaining: 1m 1s
3:	learn: 0.6711100	total: 194ms	remaining: 48.4s
4:	learn: 0.6675819	total: 206ms	remaining: 41s
5:	learn: 0.6609291	total: 217ms	remaining: 35.9s
6:	learn: 0.6568835	total: 225ms	remaining: 31.9s
7:	learn: 0.6559094	total: 231ms	remaining: 28.6s
8:	learn: 0.6499175	total: 238ms	remaining: 26.2s
9:	learn: 0.6488375	total: 244ms	remaining: 24.2s
10:	learn: 0.6452760	total: 253ms	remaining: 22.8s
11:	learn: 0.6402734	total: 259ms	remaining: 21.3s
12:	learn: 0.6367040	total: 266ms	remaining: 20.2s
13:	learn: 0.6344235	total: 272ms	remaining: 19.1s
14:	learn: 0.6323296	total: 278ms	remaining: 18.3s
15:	learn: 0.6317502	total: 285ms	remaining: 17.6s
16:	learn: 0.6298524	total: 291ms	remaining: 16.8s
17:	learn: 0.6255563	total: 299ms	remaining: 16.3s
18:	learn: 0.6237647	total: 305ms	remaining: 15.8s
19:	learn: 

168:	learn: 0.5397853	total: 1.47s	remaining: 7.21s
169:	learn: 0.5389131	total: 1.48s	remaining: 7.21s
170:	learn: 0.5387156	total: 1.49s	remaining: 7.21s
171:	learn: 0.5384450	total: 1.49s	remaining: 7.19s
172:	learn: 0.5382403	total: 1.5s	remaining: 7.18s
173:	learn: 0.5376495	total: 1.51s	remaining: 7.16s
174:	learn: 0.5373929	total: 1.52s	remaining: 7.15s
175:	learn: 0.5371367	total: 1.52s	remaining: 7.13s
176:	learn: 0.5368689	total: 1.53s	remaining: 7.11s
177:	learn: 0.5363277	total: 1.53s	remaining: 7.09s
178:	learn: 0.5360365	total: 1.54s	remaining: 7.07s
179:	learn: 0.5356932	total: 1.55s	remaining: 7.06s
180:	learn: 0.5353710	total: 1.56s	remaining: 7.04s
181:	learn: 0.5351061	total: 1.57s	remaining: 7.05s
182:	learn: 0.5348587	total: 1.58s	remaining: 7.06s
183:	learn: 0.5345480	total: 1.59s	remaining: 7.06s
184:	learn: 0.5343234	total: 1.6s	remaining: 7.04s
185:	learn: 0.5340467	total: 1.6s	remaining: 7.02s
186:	learn: 0.5336671	total: 1.61s	remaining: 7.01s
187:	learn: 0.5

337:	learn: 0.4949948	total: 2.73s	remaining: 5.35s
338:	learn: 0.4948344	total: 2.74s	remaining: 5.35s
339:	learn: 0.4946334	total: 2.75s	remaining: 5.34s
340:	learn: 0.4943995	total: 2.76s	remaining: 5.33s
341:	learn: 0.4941362	total: 2.76s	remaining: 5.32s
342:	learn: 0.4939723	total: 2.77s	remaining: 5.31s
343:	learn: 0.4938927	total: 2.78s	remaining: 5.3s
344:	learn: 0.4936690	total: 2.78s	remaining: 5.29s
345:	learn: 0.4933638	total: 2.79s	remaining: 5.28s
346:	learn: 0.4930231	total: 2.8s	remaining: 5.27s
347:	learn: 0.4927487	total: 2.81s	remaining: 5.26s
348:	learn: 0.4925658	total: 2.81s	remaining: 5.25s
349:	learn: 0.4923037	total: 2.82s	remaining: 5.24s
350:	learn: 0.4918333	total: 2.83s	remaining: 5.23s
351:	learn: 0.4916050	total: 2.84s	remaining: 5.22s
352:	learn: 0.4914072	total: 2.84s	remaining: 5.21s
353:	learn: 0.4911932	total: 2.85s	remaining: 5.2s
354:	learn: 0.4910376	total: 2.86s	remaining: 5.19s
355:	learn: 0.4908468	total: 2.87s	remaining: 5.18s
356:	learn: 0.4

510:	learn: 0.4540234	total: 4.05s	remaining: 3.87s
511:	learn: 0.4537955	total: 4.06s	remaining: 3.87s
512:	learn: 0.4535016	total: 4.07s	remaining: 3.86s
513:	learn: 0.4532656	total: 4.07s	remaining: 3.85s
514:	learn: 0.4530475	total: 4.08s	remaining: 3.84s
515:	learn: 0.4528469	total: 4.09s	remaining: 3.83s
516:	learn: 0.4526129	total: 4.1s	remaining: 3.83s
517:	learn: 0.4524377	total: 4.11s	remaining: 3.82s
518:	learn: 0.4522159	total: 4.12s	remaining: 3.82s
519:	learn: 0.4519861	total: 4.13s	remaining: 3.81s
520:	learn: 0.4516289	total: 4.13s	remaining: 3.8s
521:	learn: 0.4513876	total: 4.14s	remaining: 3.79s
522:	learn: 0.4511622	total: 4.15s	remaining: 3.78s
523:	learn: 0.4509735	total: 4.16s	remaining: 3.77s
524:	learn: 0.4508016	total: 4.16s	remaining: 3.77s
525:	learn: 0.4506437	total: 4.17s	remaining: 3.76s
526:	learn: 0.4505152	total: 4.18s	remaining: 3.75s
527:	learn: 0.4503423	total: 4.18s	remaining: 3.74s
528:	learn: 0.4501249	total: 4.19s	remaining: 3.73s
529:	learn: 0.

679:	learn: 0.4166200	total: 5.31s	remaining: 2.5s
680:	learn: 0.4164540	total: 5.32s	remaining: 2.49s
681:	learn: 0.4163127	total: 5.33s	remaining: 2.48s
682:	learn: 0.4160909	total: 5.33s	remaining: 2.48s
683:	learn: 0.4158801	total: 5.34s	remaining: 2.47s
684:	learn: 0.4157500	total: 5.35s	remaining: 2.46s
685:	learn: 0.4155831	total: 5.35s	remaining: 2.45s
686:	learn: 0.4154284	total: 5.36s	remaining: 2.44s
687:	learn: 0.4152604	total: 5.37s	remaining: 2.43s
688:	learn: 0.4150442	total: 5.37s	remaining: 2.42s
689:	learn: 0.4148629	total: 5.38s	remaining: 2.42s
690:	learn: 0.4146933	total: 5.39s	remaining: 2.41s
691:	learn: 0.4144697	total: 5.39s	remaining: 2.4s
692:	learn: 0.4142677	total: 5.4s	remaining: 2.39s
693:	learn: 0.4140512	total: 5.41s	remaining: 2.38s
694:	learn: 0.4138620	total: 5.42s	remaining: 2.38s
695:	learn: 0.4136989	total: 5.42s	remaining: 2.37s
696:	learn: 0.4134760	total: 5.43s	remaining: 2.36s
697:	learn: 0.4133199	total: 5.44s	remaining: 2.35s
698:	learn: 0.4

844:	learn: 0.3873563	total: 6.54s	remaining: 1.2s
845:	learn: 0.3872235	total: 6.55s	remaining: 1.19s
846:	learn: 0.3870101	total: 6.56s	remaining: 1.18s
847:	learn: 0.3868160	total: 6.56s	remaining: 1.18s
848:	learn: 0.3866234	total: 6.57s	remaining: 1.17s
849:	learn: 0.3864645	total: 6.58s	remaining: 1.16s
850:	learn: 0.3862068	total: 6.59s	remaining: 1.15s
851:	learn: 0.3860214	total: 6.6s	remaining: 1.15s
852:	learn: 0.3858648	total: 6.62s	remaining: 1.14s
853:	learn: 0.3857554	total: 6.62s	remaining: 1.13s
854:	learn: 0.3855850	total: 6.63s	remaining: 1.12s
855:	learn: 0.3854301	total: 6.64s	remaining: 1.12s
856:	learn: 0.3852720	total: 6.64s	remaining: 1.11s
857:	learn: 0.3850975	total: 6.66s	remaining: 1.1s
858:	learn: 0.3848917	total: 6.66s	remaining: 1.09s
859:	learn: 0.3847683	total: 6.67s	remaining: 1.08s
860:	learn: 0.3845746	total: 6.68s	remaining: 1.08s
861:	learn: 0.3844445	total: 6.69s	remaining: 1.07s
862:	learn: 0.3843024	total: 6.7s	remaining: 1.06s
863:	learn: 0.38

AUC: 0.5044317055909413


In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6418720	total: 2.47ms	remaining: 245ms
1:	learn: 0.6006486	total: 4.57ms	remaining: 224ms
2:	learn: 0.5987593	total: 6.62ms	remaining: 214ms
3:	learn: 0.5950809	total: 8.84ms	remaining: 212ms
4:	learn: 0.5922401	total: 11.1ms	remaining: 210ms
5:	learn: 0.5885374	total: 14ms	remaining: 219ms
6:	learn: 0.5831376	total: 16.3ms	remaining: 216ms
7:	learn: 0.5822347	total: 19.6ms	remaining: 225ms
8:	learn: 0.5778200	total: 21.7ms	remaining: 219ms
9:	learn: 0.5759740	total: 23.6ms	remaining: 213ms
10:	learn: 0.5701293	total: 25.8ms	remaining: 209ms
11:	learn: 0.5652587	total: 28.8ms	remaining: 211ms
12:	learn: 0.5634758	total: 31.2ms	remaining: 209ms
13:	learn: 0.5607389	total: 33.6ms	remaining: 206ms
14:	learn: 0.5538360	total: 35.6ms	remaining: 202ms
15:	learn: 0.5496386	total: 37.8ms	remaining: 199ms
16:	learn: 0.5443908	total: 41.1ms	remaining: 201ms
17:	learn: 0.5430536	total: 44.5ms	remaining: 203ms
18:	learn: 0.5

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
3         KNeighborsClassifier Precision:         0.376658
16                  Tuned LGBM Precision:         0.370866
4   Tuned KNeighborsClassifier Precision:         0.367429
6                          SVM Precision:         0.367325
8          Tuned Random Forest Precision:         0.363386
14                   Tuned XGB Precision:         0.361951
15                        LGBM Precision:         0.360223
7                Random Forest Precision:         0.359943
17                    CatBoost Precision:         0.359416
10              Tuned AdaBoost Precision:         0.358739
18              Tuned CatBoost Precision:         0.358491
13                         XGB Precision:         0.357621
12      Tuned GradientBoosting Precision:         0.357422
9                     AdaBoost Precision:         0.355874
2          Tuned Decision Tree Precision:         0.352941
11            GradientBoosting Precision:         0.3528

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
3         KNeighborsClassifier Accuracy:        0.535653
4   Tuned KNeighborsClassifier Accuracy:        0.529378
6                          SVM Accuracy:        0.503708
5                   GaussianNB Accuracy:        0.489447
0          Logistic Regression Accuracy:        0.480890
1                Decision Tree Accuracy:        0.474044
2          Tuned Decision Tree Accuracy:        0.464917
16                  Tuned LGBM Accuracy:        0.454649
13                         XGB Accuracy:        0.423274
18              Tuned CatBoost Accuracy:        0.419281
7                Random Forest Accuracy:        0.417570
8          Tuned Random Forest Accuracy:        0.415288
14                   Tuned XGB Accuracy:        0.412436
15                        LGBM Accuracy:        0.412436
9                     AdaBoost Accuracy:        0.401027
17                    CatBoost Accuracy:        0.399886
10              Tuned AdaBoost 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
16                  Tuned LGBM AUC:   0.529540
3         KNeighborsClassifier AUC:   0.516251
4   Tuned KNeighborsClassifier AUC:   0.515732
18              Tuned CatBoost AUC:   0.513291
6                          SVM AUC:   0.509861
10              Tuned AdaBoost AUC:   0.508371
9                     AdaBoost AUC:   0.506096
12      Tuned GradientBoosting AUC:   0.505421
13                         XGB AUC:   0.504452
17                    CatBoost AUC:   0.504432
8          Tuned Random Forest AUC:   0.504039
11            GradientBoosting AUC:   0.499948
14                   Tuned XGB AUC:   0.499241
15                        LGBM AUC:   0.498824
7                Random Forest AUC:   0.495183
2          Tuned Decision Tree AUC:   0.494713
1                Decision Tree AUC:   0.492440
5                   GaussianNB AUC:   0.481229
0          Logistic Regression AUC:   0.474249
